In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os.path
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import numpy as np
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [2]:
if os.path.isfile('../data/master_data.csv') and os.stat('../data/master_data.csv').st_size > 0:
    master_df = pd.read_csv('../data/master_data.csv')
else: 
    master_df = pd.DataFrame()
    
print(type(master_df))

<class 'pandas.core.frame.DataFrame'>


In [3]:
# Setup the transport and client
_transport = RequestsHTTPTransport(
    url='http://209.38.172.107/graphql/',
    use_json=True,
)

client = Client(
    transport = _transport,
    fetch_schema_from_transport=True,
)

# Define Query
query = gql('''
query TopPlayer {
  playerTotalsAll(id: 16287) {
    playerName
    position
    team
    season
    games
    assists
    blocks
    ft
    threeFg
    twoFg
    effectFgPercent
    totalRb
    steals
    points
    id
  }
  playerAdvancedAll(id: 16287) {
      playerName
    position
    team
    season
    minutesPlayed
    games
    usagePercent
    per
    tsPercent
    box
    winShares
    id
  }
}
''')

# Execute and store response

response = client.execute(query)

print(response)

{'playerTotalsAll': [{'playerName': 'Michael Jordan*', 'position': 'SG', 'team': 'CHI', 'season': 1996, 'games': 82, 'assists': 352, 'blocks': 42, 'ft': 548, 'threeFg': 111, 'twoFg': 805, 'effectFgPercent': '0.525', 'totalRb': 543, 'steals': 180, 'points': 2491, 'id': '16287'}], 'playerAdvancedAll': [{'playerName': 'Michael Jordan*', 'position': 'SG', 'team': 'CHI', 'season': 1996, 'minutesPlayed': 3090, 'games': 82, 'usagePercent': '33.300', 'per': '29.4', 'tsPercent': '0.582', 'box': '10.5', 'winShares': '20.4', 'id': '16287'}]}


In [4]:
df_totals = pd.json_normalize(response['playerTotalsAll'])
df_advanced = pd.json_normalize(response['playerAdvancedAll'])

df_combined = pd.merge(df_totals, df_advanced, on=['playerName', 'position', 'team', 'season', 'games', 'id'])

df_combined.head()

,playerName,position,team,season,games,assists,blocks,ft,threeFg,twoFg,...,totalRb,steals,points,id,minutesPlayed,usagePercent,per,tsPercent,box,winShares
0,Michael Jordan*,SG,CHI,1996,82,352,42,548,111,805,...,543,180,2491,16287,3090,33.300,29.4,0.582,10.5,20.4


In [5]:
master_df = pd.concat([master_df, df_combined], ignore_index=True)

# master_df.drop_duplicates(subset='id', keep='last', inplace=True)

master_df.to_csv('../data/master_data.csv', index=False)